In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

### 0.0 Installing Tensorflow 2.0 

In [0]:
!pip install tensorflow==2.0.0-beta0

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

### 1.0 Importing Libraries

In [0]:
import numpy as np
import tensorflow as tf
import datetime

from model import MyModel

In [2]:
tf.__version__

'2.0.0-dev20190608'

###  2.0 Importing Datasets

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [0]:
# Training Data
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

# Testing Data
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### 3.0 Importing the Model, Model Parameters & Training Hyperparameters

In [0]:
'''
Model Params
num_inputs
num_outputs
'''

model = MyModel(n_inputs, n_classes)

In [0]:
'''
learning_rate = 0.001
beta_1 = None
beta_2 = None
'''

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### 4.0 Training Step

In [0]:
@tf.function
def train_step(model, images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(model, images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

### 5.0 Training Helper Function

In [0]:
# Helper Function
def print_process(epoch, train_loss, train_accuracy, test_loss=None, test_accuracy=None):
    if test_loss!=None and test_accuracy!=None:
      template = 'Epoch {0:.0f} \tLoss: {1:.4f} \tAcc: {2:.f}% \t\tTest Loss: {3:.4f} \tTest Acc: {4:.2f}%'
      print(template.format(epoch+1,
                            train_loss.result(),
                            train_accuracy.result()*100,
                            test_loss.result(),
                            test_accuracy.result()*100))
    else:
      template = 'Epoch {0:.0f} \tLoss: {1:.4f} \tAcc: {2:.2f}%'
      print(template.format(epoch+1,
                            train_loss.result(),
                            train_accuracy.result()*100))

### 6.0 Log Directory 

Set up summary writers to write the summaries to disk in a different logs directory:

In [0]:
# Creating File Directory
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'

# Summary Writer
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

### 7.0 Training Cycle

In [0]:
'''
Use `tf.summary.scalar()` to log metrics (loss and accuracy) during training/testing 
within the scope of the summary writers to write the summaries to disk. You have control
over which metrics to log and how often to do it
'''


def train(model, train_data, test_data=None, num_epochs=5):
  
  print('Training Commenced: ')
  for epoch in range(num_epochs):
    
    for images, labels in train_data:               # Training Data
      train_step(model, images, labels)
    with train_summary_writer.as_default():
      tf.summary.scalar(name='train_loss', train_loss.result(), step=epoch)
      tf.summary.scalar(name='train_accuracy', train_accuracy.result(), step=epoch)

    if test_data!=None:
      for test_images, test_labels in test_data:    # Testing Data
        test_step(model, test_images, test_labels)
      with test_summary_writer.as_default():
        tf.summary.scalar(name='test_loss', test_loss.result(), step=epoch)
        tf.summary.scalar(name='test_accuracy', test_accuracy.result(), step=epoch)
      
    print_process(epoch, train_loss, train_accuracy, test_loss, test_accuracy)
  
  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

### 8.0 Implementation

In [26]:
train()

Training Commenced


W0609 16:18:31.807413 140279277197184 ag_logging.py:145] Entity <function train_step at 0x7f95177db840> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.contrib'
W0609 16:18:31.811664 140279277197184 ag_logging.py:145] Entity <bound method MyModel.call of <__main__.MyModel object at 0x7f951957c668>> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.contrib'


W0609 16:18:32.032044 140279277197184 ag_logging.py:145] Entity <function Function._initialize_uninitialized_variables.<locals>.initialize_variables at 0x7f95177cff28> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.contrib'
W0609 16:18:32.040365 140279277197184 ag_logging.py:145] Entity <function train_step at 0x7f95177db840> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.contrib'
W0609 16:18:32.043756 140279277197184 ag_logging.py:145] Entity <bound method MyModel.call of <__main__.MyModel object at 0x7f951957c668>> could not be transformed and will be executed as-is. Please report 

W0609 16:20:53.660938 140279277197184 ag_logging.py:145] Entity <function test_step at 0x7f95177dbe18> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.contrib'
W0609 16:20:53.664834 140279277197184 ag_logging.py:145] Entity <bound method MyModel.call of <__main__.MyModel object at 0x7f951957c668>> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.contrib'


W0609 16:20:58.451900 140279277197184 ag_logging.py:145] Entity <function test_step at 0x7f95177dbe18> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.contrib'
W0609 16:20:58.455674 140279277197184 ag_logging.py:145] Entity <bound method MyModel.call of <__main__.MyModel object at 0x7f951957c668>> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.contrib'


Epoch 1 	Loss: 0.07 	Acc: 97.76% 		Test Loss: 0.06 	Test Acc: 98.13%
Epoch 2 	Loss: 0.05 	Acc: 98.43% 		Test Loss: 0.06 	Test Acc: 98.20%
Epoch 3 	Loss: 0.04 	Acc: 98.80% 		Test Loss: 0.06 	Test Acc: 98.26%
Epoch 4 	Loss: 0.03 	Acc: 99.01% 		Test Loss: 0.06 	Test Acc: 98.26%
Epoch 5 	Loss: 0.03 	Acc: 99.15% 		Test Loss: 0.07 	Test Acc: 98.25%


### 9.0 Tensorboard

In [0]:
%tensorboard --logdir logs/gradient_tape

### Code Reference

- https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/get_started.ipynb#scrollTo=-Iue509kgOyE
- https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/quickstart/advanced.ipynb